# Net2Graph

In [6]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship, Subgraph, NodeMatcher
from graphdatascience import GraphDataScience
from tqdm import tqdm
import json
import joblib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Network Embedding

In [7]:
#neo4j DB info
#7473/tcp, 0.0.0.0:7471->7474/tcp, 0.0.0.0:7681->7687/tcp
db_info = {
    'uri' : "bolt://10.109.6.14:7681",
    'user' : "neo4j",
    'password' : "admin",
    }
graph_db = Graph(db_info['uri'], user=db_info['user'], password=db_info['password'])
gds = GraphDataScience(db_info['uri'], auth=(db_info['user'], db_info['password']))


In [11]:
nodes = ["Doc", "User", "Keyword"]
relationships = {"HAS_KEYWORD": {"orientation": "UNDIRECTED", "properties": "score"}, "LIKE": {"orientation": "UNDIRECTED", "properties": "clicked"}}
#project
G, _ = gds.graph.project("tmp_graph", nodes, relationships)
#fastRP
result = gds.fastRP.write(
    G,
    writeProperty='embedding',
    embeddingDimension=128,
    iterationWeights=[0.8, 1, 1, 1],
    )
G.drop();


## Similarity

In [20]:
cypher = f"""
MATCH (a)-[r:SIMILAR_USER]-()
DELETE r
"""
gds.run_cypher(cypher)

node_projection = ["User"]
relationship_projection = {"LIKE": {"orientation": "UNDIRECTED"}}
nodeProperties=['embedding']
G, _ = gds.graph.project("tmp_graph", node_spec=node_projection, relationship_spec=relationship_projection, nodeProperties=nodeProperties)

_ = gds.knn.write(
    G, topK=5, nodeProperties=["embedding"],
    writeRelationshipType="SIMILAR_USER", writeProperty="score",
)

G.drop();

In [21]:
cypher = f"""
MATCH (a)-[r:SIMILAR_DOC]-()
DELETE r
"""
gds.run_cypher(cypher)

node_projection = ["Doc"]
relationship_projection = {"LIKE": {"orientation": "UNDIRECTED"}}
nodeProperties=['embedding']
G, _ = gds.graph.project("tmp_graph", node_spec=node_projection, relationship_spec=relationship_projection, nodeProperties=nodeProperties)

_ = gds.knn.write(
    G, topK=100, nodeProperties=["embedding"],
    writeRelationshipType="SIMILAR_DOC", writeProperty="score",
)

G.drop();

Knn:   0%|          | 0/100 [00:00<?, ?%/s]